In [1]:
import pandas as pd
import numpy as np

train = []
labels = []

dfPredict = pd.read_csv("train.csv", header = 0, low_memory=False)
dfBackground = pd.read_csv("noConstants.csv", header = 0, low_memory=False)
# GPA
rows = dfPredict.shape[0]
for i in range(rows):

    score = unicode("layoff", "utf-8")
    key = unicode("challengeID", "utf-8")
    gpa = dfPredict.loc[dfPredict.index[[i]], score]
    labels.append(gpa.values)
    
    # Gets corresponding data for each challengeID in training set
    challengeID = dfPredict.loc[dfPredict.index[[i]], key].values
    data = dfBackground.loc[dfBackground.index[[challengeID - 1]]]
    
    # Removes from the array the id
    withoutID = data.drop(["challengeID"], axis=1)
    
    train.append(withoutID.values)


In [2]:
import math
newTrain = []
newLabels = []

for i in range(len(labels)):
    if not math.isnan(labels[i][0]):
        newTrain.append(train[i][0])
        newLabels.append(labels[i][0])


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
transformTrain = []
tobin = []
stringIndices = []

for arr in newTrain:
    for each in arr:
        if type(each) is str:
            tobin.append(each)

trans = le.fit_transform(tobin)

count = 0
for arr in newTrain:
    for each in arr:
        if type(each) is str:
            y = trans[count]
            count = count + 1
            transformTrain.append(y)
        else:
            transformTrain.append(each)
transformTrain = np.reshape(np.array(transformTrain), (len(newTrain),-1))


# for arr in newTrain:
#     for each in arr:
#         if type(each) is str:
#             m = re.search("\d", each)
#             if m:
#                 y = float(each[m.start()])
#             else:
#                 y = 0.0
#             transformTrain.append(y)
#         else:
#             transformTrain.append(each)
# transformTrain = np.reshape(np.array(transformTrain), (len(newTrain),-1))


In [5]:
import re
x = '8-9'
m = re.search("\d", x)
x[m.start()]

'8'

In [68]:
gpas = {1.0: 0, 1.25: 1, 1.50: 2, 1.75: 3, 2.0: 4, 2.25: 5, 2.50:6, 2.75:7, 3.0:8, 3.25:9, 3.50:10, 3.75: 11, 4.0:12}

In [69]:
classLabels = []
for each in newLabels:
    classLabels.append(gpas[each])

In [4]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(transformTrain, newLabels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
lr.score(transformTrain, newLabels)

0.9420516836335161

In [6]:
# prediction for all samples

rows = dfBackground.shape[0]
allSamples = []
for i in range(rows):
    
    data = dfBackground.loc[dfBackground.index[[i]]]
    allSamples.append(data.drop(["challengeID"], axis=1).values[0])
    


In [7]:
transformSamples = []


tobin = []
for arr in allSamples:
    for each in arr:
        if type(each) is str:
            tobin.append(each)
#             y = le.fit_transform(each)
#             transformTrain.append(y)

trans = le.fit_transform(tobin)

count = 0
for arr in allSamples:

    for each in arr:
        if type(each) is str:
            y = trans[count]
            count = count + 1
            transformSamples.append(y)
        else:
            transformSamples.append(each)
transformSamples = np.reshape(np.array(transformSamples), (len(allSamples),-1))


# for arr in allSamples:
#     for each in arr:
#         if type(each) is str:
#             m = re.search("\d", each)
#             if m:
#                 y = float(each[m.start()])
#             else:
#                 y = 0.0
#             transformSamples.append(y)
#         else:
#             transformSamples.append(each)
# transformSamples = np.reshape(np.array(transformSamples), (len(allSamples),-1))
print len(transformSamples[0])

10594


In [13]:
out = lr.predict_proba(transformSamples)
# inv_gpas = {v: k for k, v in gpas.iteritems()}
# transOut = []
# for each in out:
#     transOut.append(inv_gpas[each])


In [9]:
dfResults = pd.read_csv("prediction.csv", header = 0, low_memory=False)

In [14]:
dfResults["layoff"] = np.array(out) #transOut)

In [15]:
dfResults.to_csv('prediction.csv', index=False)

In [12]:
out

array([ 0.,  1.,  0., ...,  0.,  1.,  0.])